In [1]:
import pandas as pd
import numpy as np
np.random.seed(42)
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
import re
from collections import defaultdict
import requests
N = 5
raw_meta = pd.read_csv('gene_data/pe-rna-metadata.csv',header=None, names=["sample", "classification"]).T
catagorical_ground_truth  = pd.DataFrame(raw_meta.values[1:], columns=raw_meta.iloc[0])
catagorical_ground_truth.columns.name = None
ground_truth = [{"Control": "ctrl", "Severe": "case", "Mild": "case"}.get(item, item) for item in list(catagorical_ground_truth.loc[0])]
to_name = pd.read_csv('gene_data/genes.csv').set_index("code")["name"].to_dict()
to_ensembl = pd.read_csv('gene_data/genes.csv').set_index("name")["code"].to_dict()
unfiltered_counts = np.log1p(pd.read_csv('gene_data/pe-rna-counts.csv', index_col=0))
codes = [g for g in list(pd.read_csv('gene_data/genes.csv')["code"]) if g in unfiltered_counts.index]
counts = unfiltered_counts.loc[list(set(codes))]
genes = list(set([to_name[x] for x in codes]))


d = {}
for g in tqdm(genes):
    d[g] = re.findall(
        r'"name":"(.*?)","dataSource":',
        requests.get(
            "https://www.pathwaycommons.org/pc2/top_pathways?q=" + g
        ).text,
    )

pathways_to_genes = defaultdict(list)
for gene, pathways in d.items():
    for pathway in pathways:
        pathways_to_genes[pathway].append(gene)
pathways_to_genes = dict(pathways_to_genes)

100%|██████████| 73/73 [00:18<00:00,  4.03it/s]


### Which pathways are most important in the logistic regression?

In [2]:
logreg = LogisticRegression(tol=1e-8, random_state=5)
logreg.fit(counts.T, ground_truth)
coef = pd.DataFrame(logreg.coef_[0], index=counts.index).T
coef.columns = [to_name[x] for x in coef.columns]

To determine which pathways are the most signficant, we run through each pathway and get the mean nonnegative coefficent for all the genes in the pathway

Then we print the ten most signficant pathways

In [3]:
d = {}
for pathway, genes in pathways_to_genes.items():
    d[pathway] = np.mean([coef[gene].values[0] for gene in genes if coef[gene].values[0] >=0])
top_pathways = sorted(d, key=d.get, reverse=True)[:10]
top_pathways

/home/punky/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/punky/anaconda3/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


['IGF1 pathway',
 'IGF1 signaling pathway ( IGF1 signaling pathway )',
 'Stabilization and expansion of the E-cadherin adherens junction',
 'Ceramide signaling pathway',
 'Developmental Biology',
 'Plasma membrane estrogen receptor signaling',
 'SHP2 signaling',
 'Insulin/IGF pathway-protein kinase B signaling cascade',
 'Insulin/IGF pathway-mitogen activated protein kinase kinase/MAP kinase cascade',
 'Integrins in angiogenesis']

And these are the genes which corespond to the pathways

In [4]:
[pathways_to_genes[path] for path in top_pathways]

[['IGF1'],
 ['IGF1'],
 ['IGF1'],
 ['IGF1', 'TNF'],
 ['IGF1'],
 ['IGF1', 'NOS3'],
 ['IGF1', 'NOS3', 'VEGFA'],
 ['IGF1', 'IGF2', 'IGF2R'],
 ['IGF1', 'IGF2', 'IGF2R'],
 ['IGF1', 'FN1', 'CSF1', 'VEGFA', 'ITGB3']]